In [ ]:
# Файл
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

### Задание 1

In [ ]:
def count_success_and_failure(file_path):
    with open(file_path, encoding='utf-8') as f:
        lines = f.readlines()

    result_dict = {'total_errors':0, 'subscription_renewal':0}

    for i, line in enumerate(lines):
        if 'ERROR' in line:
            result_dict['total_errors'] = result_dict.get('total_errors',0) + 1
        if 'Обновляем подписку пользователю id:' in line:
            if i == len(lines)-1 or 'ошибка при списании' not in lines[i+1]:
                result_dict['subscription_renewal'] = result_dict.get('subscription_renewal',0) + 1

    return (result_dict['subscription_renewal'], result_dict['total_errors'])

### Задание 2

In [ ]:
# Загружаем библиотеки
import re

In [ ]:
def auto_renewal_sub(log_file_path):
    with open(log_file_path, encoding='utf-8') as f:
        lines = f.readlines()
    
    # Формируем словарь, где ключи — это уникальные даты, а значения — пустой список.
    dates_dict = {re.findall(r'\d{4}-\d{2}-\d{2}', line)[0]:[] for line in lines}
    
    # Добавляем значение строки "количество людей с автопродлением подписки: ..." в список соответствующей даты.
    for line in lines:
        date = re.findall(r'\d{4}-\d{2}-\d{2}', line)[0]
        if 'количество людей с автопродлением подписки' in line and date in dates_dict:
            for part in line.split(" | ")[-1].split(" "):
                for el in part.replace("\n","").split(" "):
                    if el.isdigit():
                        dates_dict[date].append(int(el))
    
    # Если в какую-либо дату было несколько сообщений о количестве продлений, берем максимальное значение.
    unique_dates_updated = {k:max(v) for k, v in dates_dict.items()}
    unique_dates_list = list(unique_dates_updated.values())

    # Рассчитываем сглаженные значения. 
    running_avg = [round(sum(unique_dates_list[:(i+1)])/len(unique_dates_list[:(i+1)]),2) for i, j in enumerate(unique_dates_list)]
    running_median = [sorted(unique_dates_list[:(i+1)], reverse=True)[len(unique_dates_list[:(i+1)])//2] for i, j in enumerate(unique_dates_list)]

    # Записываем в файл.
    with open('auto_renewal_sub.txt', mode='w') as file:
        file.write(f'Среднее: {running_avg}\nМедиана: {running_median}')

### Задание 3

In [ ]:
# Загружаем библиотеки
import re
from datetime import datetime

In [ ]:
def sub_renewal_by_day(file_path):
    with open(file_path, encoding='utf-8') as f:
        lines = f.readlines()

    result_dict = {}
    
    # Формируем словарь, где ключи — это даты продления подписки, а значения — количество успешных продлений.
    for i, line in enumerate(lines):
        if 'Обновляем подписку пользователю id:' in line:
            if i == len(lines)-1 or 'ошибка при списании' not in lines[i+1]:
                pattern = re.compile(r"\d{4}-\d{2}-\d{2}")
                date = pattern.search(line).group()
                result_dict[date] = result_dict.get(date,0) + 1
    
    # Переводим даты в дни недели и суммируем количество успешных продлений по каждому дню недели.
    week_days = {"Monday": 0,
            "Tuesday":0,
            "Wednesday":0,
            "Thursday":0,
            "Friday":0,
            "Saturday":0,
            "Sunday":0}
    for k,v in result_dict.items():
        if datetime.strptime(k, "%Y-%m-%d").strftime('%A') in week_days:
            week_days[datetime.strptime(k, "%Y-%m-%d").strftime('%A')] += result_dict[k]

    # Формируем словарь и заменяем англоязычные названия дней недели на русскоязычные.
    weekDaysMapping = {"Monday": "Понедельник",
                   "Tuesday":"Вторник",
                   "Wednesday":"Среда",
                   "Thursday":"Четверг",
                   "Friday":"Пятница",
                   "Saturday":"Суббота",
                   "Sunday":"Воскресенье"}

    dates_to_day = {weekDaysMapping.get(k):v for k,v in week_days.items()}

    # Преобразуем данные в требуемый формат.
    res_data = ['Количество обновлений подписки по дням недели:']
    for key, values in dates_to_day.items():
        res_data.append(f'{key}: {values}')

    # Записываем данные в файл.
    with open('weekdays.txt', mode='w', newline='') as file:
        file.writelines('\n'.join(res_data))